In [3]:
import pandas as pd


In [ ]:
from pathlib import Path
print("CWD:", Path.cwd())

In [57]:
df_task_dwa = pd.read_csv("../data/raw/Tasks to DWAs.txt", sep="\t", dtype=str, na_filter=False)
df_task_desc = pd.read_csv("../data/raw/Task Statements.txt", sep="\t", dtype=str, na_filter=False)
df_dwa_desc = pd.read_csv("../data/raw/DWA Reference.txt", sep="\t", dtype=str, na_filter=False)
df_occ = pd.read_csv("../data/raw/Occupation Data.txt", sep="\t", dtype=str, na_filter=False)



In [58]:
df_task_dwa.columns = (df_task_dwa.columns
              .str.strip()
              .str.lower()
              .str.replace(r"\s+", "_", regex=True))
to_drop = ["date", "domain_source"]
df_task_dwa = df_task_dwa.drop(columns=to_drop)
print(df_task_dwa.columns)

Index(['o*net-soc_code', 'task_id', 'dwa_id'], dtype='object')


In [59]:
df_task_desc.columns = (df_task_desc.columns
                      .str.strip()
                      .str.lower()
                      .str.replace(r"\s+", "_", regex=True))

print(df_task_desc.columns)

Index(['o*net-soc_code', 'task_id', 'task', 'task_type',
       'incumbents_responding', 'date', 'domain_source'],
      dtype='object')


In [60]:
df_dwa_desc.columns = (df_dwa_desc.columns
                      .str.strip()
                      .str.lower()
                      .str.replace(r"\s+", "_", regex=True))
print(df_dwa_desc.columns)

Index(['element_id', 'iwa_id', 'dwa_id', 'dwa_title'], dtype='object')


In [61]:
df_occ.columns = (df_occ.columns
                      .str.strip()
                      .str.lower()
                      .str.replace(r"\s+", "_", regex=True))
df_occ = df_occ.rename(columns={"title": "occ_title", "description": "occ_description"})
print(df_occ.columns)

Index(['o*net-soc_code', 'occ_title', 'occ_description'], dtype='object')


In [62]:
for df, cols in [
    (df_task_dwa, ['o*net-soc_code','task_id','dwa_id']),
    (df_occ,      ['o*net-soc_code','occ_title','occ_description']),
    (df_dwa_desc, ['dwa_id','dwa_title']),
    (df_task_desc,['task_id','task'])
]:
    missing = [c for c in cols if c not in df.columns]
    if missing: raise ValueError(f"Missing {missing} in dataframe")

In [63]:
#check for dupes
assert not df_occ.duplicated('o*net-soc_code').any(), "Duplicates in df_occ on o*net-soc_code"
assert not df_dwa_desc.duplicated('dwa_id').any(),     "Duplicates in df_dwa_desc on dwa_id"
assert not df_task_desc.duplicated('task_id').any(),   "Duplicates in df_task_desc on task_id"



In [64]:
df = (
    df_task_dwa
      .merge(
          df_occ[['o*net-soc_code','occ_title','occ_description']],
          on='o*net-soc_code', how='left', validate='many_to_one'
      )
      .merge(
          df_dwa_desc[['dwa_id','dwa_title']],
          on='dwa_id', how='left', validate='many_to_one'
      )
      .merge(
          df_task_desc[['task_id','task']],
          on='task_id', how='left', validate='many_to_one'
      )
)

print(df.shape)


(23851, 7)


In [65]:
df.head()

,o*net-soc_code,task_id,dwa_id,occ_title,occ_description,dwa_title,task
0,11-1011.00,20461,4.A.2.a.4.I09.D03,Chief Executives,Determine and formulate policies and provide o...,Analyze impact of legal or regulatory changes.,"Review and analyze legislation, laws, or publi..."
1,11-1011.00,20461,4.A.4.b.6.I08.D04,Chief Executives,Determine and formulate policies and provide o...,Advise others on legal or regulatory complianc...,"Review and analyze legislation, laws, or publi..."
2,11-1011.00,8823,4.A.4.b.4.I09.D02,Chief Executives,Determine and formulate policies and provide o...,Direct financial operations.,Direct or coordinate an organization's financi...
3,11-1011.00,8824,4.A.4.a.2.I03.D14,Chief Executives,Determine and formulate policies and provide o...,Confer with organizational members to accompli...,"Confer with board members, organization offici..."
4,11-1011.00,8825,4.A.2.a.4.I07.D09,Chief Executives,Determine and formulate policies and provide o...,Analyze data to assess operational or project ...,Analyze operations to evaluate performance of ...


In [66]:
df.sample(3)

,o*net-soc_code,task_id,dwa_id,occ_title,occ_description,dwa_title,task
13945,35-2012.00,9491,4.A.3.a.1.I08.D03,"Cooks, Institution and Cafeteria",Prepare and cook large quantities of food for ...,Clean tableware.,"Wash pots, pans, dishes, utensils, or other co..."
2226,13-1141.00,3363,4.A.2.a.1.I02.D04,"Compensation, Benefits, and Job Analysis Speci...",Conduct programs of compensation and benefits ...,Evaluate effectiveness of personnel policies o...,Research employee benefit and health and safet...
11824,29-1299.02,18425,4.A.4.a.5.I11.D05,Orthoptists,Diagnose and treat visual system disorders suc...,Assist healthcare practitioners during examina...,Assist ophthalmologists in diagnostic ophthalm...


In [67]:
df.isna().sum()

o*net-soc_code     0
task_id            0
dwa_id             0
occ_title          0
occ_description    0
dwa_title          0
task               0
dtype: int64

In [68]:
df["task_len"] = df["task"].str.split().str.len()
df["task_len"].describe()

count    23851.000000
mean        13.773301
std          5.508161
min          2.000000
25%         10.000000
50%         13.000000
75%         18.000000
max         45.000000
Name: task_len, dtype: float64

In [72]:
tasks_per_dwa = df.groupby("dwa_id")["task_id"].nunique().sort_values(ascending=False)
print(tasks_per_dwa)


dwa_id
4.A.4.b.6.I07.D02    124
4.A.2.a.1.I03.D04    115
4.A.3.b.6.I08.D03    111
4.A.2.b.3.I01.D10     86
4.A.3.b.6.I11.D04     76
                    ... 
4.A.3.a.2.I45.D04      1
4.A.2.b.2.I26.D04      1
4.A.3.b.4.I05.D05      1
4.A.1.b.2.I10.D10      1
4.A.4.c.2.I01.D07      1
Name: task_id, Length: 2083, dtype: int64


In [73]:
dwas_per_task = df.groupby("task_id")["dwa_id"].nunique().value_counts().sort_index()
print(dwas_per_task)

dwa_id
1    14431
2     3692
3      661
4       12
5        1
Name: count, dtype: int64


In [75]:
dup_rows = df.duplicated(subset=["task_id", "dwa_id"]).sum()
print(dup_rows)

0


In [76]:
dup_tasks = df.duplicated(subset=["task"]).sum()
print(dup_tasks)

6313


In [77]:
df.columns


Index(['o*net-soc_code', 'task_id', 'dwa_id', 'occ_title', 'occ_description',
       'dwa_title', 'task', 'task_len'],
      dtype='object')